In [ ]:
import requests
from bs4 import BeautifulSoup
import logging
from transformers import pipeline
import re
import logging
from keybert import KeyBERT
from typing import Dict,List,Set
import os


i:\project\Assigment A\newsa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
log_file = "app.log"

# Ensure log file exists
if not os.path.exists(log_file):
    open(log_file, "w").close()

# Remove previous handlers to allow reconfiguration
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filename=log_file,
    filemode="a",
    force=True
)

In [6]:
# Load Summary model 
summary_pipeline = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
# Load Sentiment model 
sentiment_pipeline = pipeline(
                                "sentiment-analysis",
                                model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
                                )
#Load keyBERT for Topic Modelling for every articale
kw_model = KeyBERT()

Device set to use cpu
Device set to use cpu


In [18]:


def extract_articles(company: str) -> List[Dict[str, str]]:
    """Extracts the latest news articles for a given company."""
    logging.info(f"Starting article extraction for company: {company}")
    headers = {
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                       '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    }
    url = f"https://timesofindia.indiatimes.com/topic/{company}"
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        logging.error(f"Failed to fetch the main page: {e}")
        return []
    
    soup = BeautifulSoup(response.content, "html.parser")
    articles = []
    article_divs = soup.select('div.uwU81')[:2]
    
    for div in article_divs:
        a_tag = div.find('a')
        if not a_tag or 'href' not in a_tag.attrs:
            continue
        
        article_url = a_tag['href']
        
        try:
            request_internal = requests.get(article_url, headers=headers)
            request_internal.raise_for_status()
        except requests.RequestException as e:
            logging.warning(f"Skipping article due to request error: {e}")
            continue
        
        soup_internal = BeautifulSoup(request_internal.content, "html.parser")
        title_tag = soup_internal.find('h1')
        title = title_tag.get_text(strip=True) if title_tag else "No Title Found"
        
        content_div = soup_internal.find('div', class_=re.compile(r'_s30J\s*clearfix'))
        content = ''
        if content_div:
            paragraphs = content_div.find_all('p')
            content = '\n'.join([p.text.strip() for p in paragraphs]) if paragraphs else content_div.text.strip()
        
        if not content:
            logging.info(f"Skipping article '{title}' due to missing content.")
            continue
        
        summary_text = summary_pipeline(content, max_length=80, min_length=20, do_sample=False)
        summary = summary_text[0]['summary_text']
        
        label = sentiment_pipeline(summary)[0] if summary else {'label': "NEUTRAL", 'score': 0.5}
        
        topics = kw_model.extract_keywords(summary, keyphrase_ngram_range=(1, 2), stop_words='english')
        topic_list = [topic[0] for topic in topics]
        
        articles.append({
            "Title": title,
            "Summary": summary,
            "Sentiment": label,
            "URL": article_url,
            "Topics": topic_list
        })
    
    return articles

In [19]:
articles=extract_articles('tesla')
print(articles)

[{'Title': 'Tesla hit by record trade-ins as Elon Musk’s DOGE ties sparks outrage', 'Summary': " A significant increase in Tesla vehicle trade-ins due to protests against CEO Elon Musk's political involvement . Demonstrations and vandalism have erupted, causing a decline in Tesla's sales . Tesla cars from 2017 and newer made up 1.4% of all vehicles traded in until March 15 . Analysts at Edmunds believe this number could rise further in the coming weeks .", 'Sentiment': {'label': 'NEGATIVE', 'score': 0.9929310083389282}, 'URL': 'https://timesofindia.indiatimes.com/technology/tech-news/tesla-hit-by-record-trade-ins-as-elon-musks-doge-ties-sparks-outrage/articleshow/119311941.cms', 'Topics': ['tesla sales', 'sales tesla', 'tesla cars', 'increase tesla', 'tesla vehicle']}, {'Title': "Jasmine Crockett wants 'Elon Musk to be taken down' as her birthday present amid violence against Tesla", 'Summary': ' Congresswoman Jasmine Crockett (D-TX) said she wants Elon Musk to be taken down as her bir

In [20]:
def comparative_analysis(articles: List[Dict[str, any]], company_name: str) -> Dict[str, any]:
    logging.info(f" comparative analysis of news articles related to: {company_name}")
    all_topic_sets: List[Set[str]] = [set(article["Topics"]) for article in articles]
    common_topics: Set[str] = set.intersection(*all_topic_sets) if all_topic_sets else set()

    comparisons = []
    for i in range(len(articles) - 1):
        for j in range(i + 1, len(articles)):
            art1, art2 = articles[i], articles[j]
            comparisons.append({
                "Comparison": f"{art1['Title']} vs {art2['Title']}",
                "Impact": (
                    f"{art1['Title']} discusses {art1['Sentiment']['label']} news, whereas "
                    f"{art2['Title']} focuses on {art2['Sentiment']['label']} coverage."
                )
            })
    
    topic_overlap = {
        "Common Topics": list(common_topics),
        "Unique Topics per Article": {
            art["Title"]: list(set(art["Topics"]) - common_topics) for art in articles
        }
    }
    
    sentiment_counts = {"POSITIVE": 0, "NEGATIVE": 0, "NEUTRAL": 0}
    for art in articles:
        sentiment_counts[art['Sentiment']['label']] += 1
    
    return {
        "Company": company_name,
        "Articles": articles,
        "Comparative Sentiment Score": {
            "Sentiment Distribution": sentiment_counts,
            "Coverage Differences": comparisons,
            "Topic Overlap": topic_overlap
        },
        "Final Sentiment Analysis": (
            f"{company_name}'s latest news coverage is mostly "
            f"{max(sentiment_counts, key=sentiment_counts.get)}."
        )
    }

In [21]:
result = comparative_analysis(articles,'Tesla')
result


{'Company': 'Tesla',
 'Articles': [{'Title': 'Tesla hit by record trade-ins as Elon Musk’s DOGE ties sparks outrage',
   'Summary': " A significant increase in Tesla vehicle trade-ins due to protests against CEO Elon Musk's political involvement . Demonstrations and vandalism have erupted, causing a decline in Tesla's sales . Tesla cars from 2017 and newer made up 1.4% of all vehicles traded in until March 15 . Analysts at Edmunds believe this number could rise further in the coming weeks .",
   'Sentiment': {'label': 'NEGATIVE', 'score': 0.9929310083389282},
   'URL': 'https://timesofindia.indiatimes.com/technology/tech-news/tesla-hit-by-record-trade-ins-as-elon-musks-doge-ties-sparks-outrage/articleshow/119311941.cms',
   'Topics': ['tesla sales',
    'sales tesla',
    'tesla cars',
    'increase tesla',
    'tesla vehicle']},
  {'Title': "Jasmine Crockett wants 'Elon Musk to be taken down' as her birthday present amid violence against Tesla",
   'Summary': ' Congresswoman Jasmine C